In [36]:
import pandas as pd
import os 
import csv
import re
data=[]
directory=r'C:\Users\mvvha\OneDrive\Desktop\soc\aclImdb_v1\aclImdb\train\pos'
for i in os.listdir(directory):
    if i.endswith(".txt"):
         file_path = os.path.join(directory,i)
         with open(file_path, 'r', encoding='utf-8') as file:
              content=file.read()
              data.append(content)
rev=["positive"]*len(data)
pos_data={"Review":data,"sentiment":rev}
positive_data=pd.DataFrame(pos_data)
data=[]
directory=r'C:\Users\mvvha\OneDrive\Desktop\soc\aclImdb_v1\aclImdb\train\neg'
for i in os.listdir(directory):
    if i.endswith(".txt"):
         file_path = os.path.join(directory,i)
         with open(file_path, 'r', encoding='utf-8') as file:
              content=file.read()
              data.append(content)
rev=["negative"]*len(data)
neg_data={"Review":data,"sentiment":rev}
negative_data=pd.DataFrame(neg_data)
csv_file_path_1 = './positive.csv'
csv_file_path_2= './negative.csv'
positive_data.to_csv(csv_file_path_1, index=False)
negative_data.to_csv(csv_file_path_2,index=False)
print("no. of missing data in pos_data",positive_data['Review'].isnull().sum())
print("no. of missing data in neg_data",negative_data['Review'].isnull().sum())
positive_data['Review']=positive_data['Review'].str.lower()
negative_data['Review']=negative_data['Review'].str.lower()
def remove_html_tags(text):
     pattern=re.compile('<,*>')
     return pattern.sub(r' ',text)
positive_data['Review']=positive_data['Review'].apply(remove_html_tags)
negative_data['Review']=negative_data['Review'].apply(remove_html_tags)
import string 
punch=string.punctuation
def remove_punc(text):
     return text.translate(str.maketrans(" "," ",punch))
positive_data['Review']=positive_data['Review'].apply(remove_punc)
negative_data['Review']=negative_data['Review'].apply(remove_punc)
def remove_extra_white(text):
     cleaned_line = re.sub(r'\s+', ' ', text).strip()
     return cleaned_line
positive_data['Review']=positive_data['Review'].apply(remove_extra_white)
negative_data['Review']=negative_data['Review'].apply(remove_extra_white)


no. of missing data in pos_data 0
no. of missing data in neg_data 0


In [37]:
from nltk.corpus import stopwords
def tokenize_data(text):
    tokens=text.split(" ")
    return tokens
positive_data['Review']=positive_data['Review'].apply(tokenize_data)
negative_data['Review']=negative_data['Review'].apply(tokenize_data)
def remove_stop_words(text):
    stop_words=stopwords.words("english")
    text_out=[]
    for word in text:
        if word not in stop_words:
            text_out.append(word)
    return text_out
positive_data['Review']=positive_data['Review'].apply(remove_stop_words)
negative_data['Review']=negative_data['Review'].apply(remove_stop_words)
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def lemmatize_data(text):
    z=[]
    for word in text:
        out=lemmatizer.lemmatize(word,pos="v")
        z.append(out)
    return z
# text = ["running", "ate", "better", "cats"]
# lemmatized_text = lemmatize_data(text)
# print(lemmatized_text)
positive_data['Review']=positive_data['Review'].apply(lemmatize_data)
negative_data['Review']=negative_data['Review'].apply(lemmatize_data)
print(positive_data['Review'].sample(n=2))
positive_data["cleaned_review"]=positive_data['Review'].apply(lambda x:" ".join(x))
negative_data['cleaned_review']=negative_data['Review'].apply(lambda x:" ".join(x))
print(positive_data['cleaned_review'].sample(n=2))

7219    [okay, like, many, film, spawn, snl, skit, tim...
4232    [film, leave, feel, gregor, jordans, ned, kell...
Name: Review, dtype: object
6610    finish watch movie maybe 7th 8th time pick one...
9489    typically watch short film always afraid perso...
Name: cleaned_review, dtype: object


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
df_pos = pd.DataFrame(positive_data['cleaned_review'])
df_neg=pd.DataFrame(negative_data['cleaned_review'])
positive_data['label']=[1]*len(positive_data)
negative_data['label']=[0]*len(negative_data)
net_df = pd.concat([positive_data,negative_data], ignore_index=True)
vectorizer = TfidfVectorizer()
x_train= vectorizer.fit_transform(net_df['cleaned_review'])
print(net_df['cleaned_review'].sample(n=2))

13271    ok read film somewhere internet many criticize...
20704    could say better gymkata least felt money tota...
Name: cleaned_review, dtype: object
Document-Term Matrix (TF-IDF):

Vocabulary Mapping (Word to Index):


In [39]:
pos_directory = r'C:\Users\mvvha\OneDrive\Desktop\soc\aclImdb_v1\aclImdb\test\pos'
neg_directory = r'C:\Users\mvvha\OneDrive\Desktop\soc\aclImdb_v1\aclImdb\test\neg'
def load_data_from_directory(directory, sentiment):
    data = []
    for i in os.listdir(directory):
        if i.endswith(".txt"):
            file_path = os.path.join(directory, i)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                data.append(content)
    labels = [1 if sentiment == "positive" else 0] * len(data)
    return pd.DataFrame({"Review": data, "sentiment": sentiment, "label": labels})

# Load positive and negative test data
positive_data_test = load_data_from_directory(pos_directory, "positive")
negative_data_test = load_data_from_directory(neg_directory, "negative")
# Combine positive and negative test data
net_test = pd.concat([positive_data_test, negative_data_test], ignore_index=True)
net_test["Review"]=net_test["Review"].apply(remove_html_tags)
net_test["Review"]=net_test["Review"].apply(remove_punc)
net_test["Review"]=net_test["Review"].apply(remove_extra_white)
net_test["Review"]=net_test["Review"].apply(tokenize_data)
net_test["Review"]=net_test["Review"].apply(remove_stop_words)
net_test["Review"]=net_test["Review"].apply(lemmatize_data)
net_test["clean_review"]=net_test["Review"].apply(lambda x: " ".join(x))

# Transform the test data using the fitted vectorizer
X_test = vectorizer.transform(net_test['clean_review'])

# Get the labels
y_train = net_df['label']
y_test = net_test['label']

# Initialize Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier on the training data
nb_classifier.fit(x_train, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Naive Bayes Classifier: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy of Naive Bayes Classifier: 0.84

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84     12500
           1       0.84      0.82      0.83     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000

